In [1]:
wd = "/Users/marcinsawinski/Documents/GitHub/factue-task2"
import sys, os

os.chdir(wd)

In [2]:
import duckdb

df = duckdb.read_parquet("data/cleaned/normalization/**/*.parquet")

In [3]:
duckdb.sql("SELECT COUNT(*) FROM 'data/cleaned/normalization/**/*.parquet'")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         7829 │
└──────────────┘

In [17]:
import pandas as pd
from pathlib import Path

root_dir = Path("data/llm_output/normalization/extract")
files = list(root_dir.rglob("**/v002/*/dev/**/*.parquet")) #+ list(
#     root_dir.rglob("**/train/**/*.parquet")
# )

# Read and tag each file with its source path in one step
all_dfs = []
for f in files:
    # print(f)
    try:
        df = pd.read_parquet(f)
        df["source_file"] = f
        all_dfs.append(df)
    except Exception as e:
        print(f"Failed to read {f}: {e}")

# Concatenate all DataFrames
combined_df = pd.concat(all_dfs, ignore_index=True)

In [18]:
combined_df.max_iterations = combined_df.max_iterations.astype(str)

In [19]:
pivot_df = combined_df.pivot_table(
    index=[
        "lang",
        "prompt_version",
    ],
    columns=["model_name", "max_iterations"],
    values="text",
    aggfunc="count",
)
# pivot_df = pivot_df[["train", "dev", "test"]]
# pivot_df["zero"] = pivot_df.train.isna()
# pivot_df = (
#     pivot_df.sort_values(["zero", "train"], ascending=[True, False])
#     .drop(columns="zero")
#     .reset_index()
# )
pivot_df

,model_name,deepseek-r1:8b,gpt-4.1-mini,gpt41-mini-ft-all-norm
,max_iterations,3,1,1
lang,prompt_version,,,
dev-ara,v002,61.0,NaN,61.0
dev-deu,v002,32.0,NaN,32.0
dev-eng,v002,100.0,100.0,200.0
dev-fra,v002,53.0,53.0,53.0
dev-hi,v002,17.0,NaN,17.0
dev-mr,v002,21.0,NaN,21.0
dev-msa,v002,20.0,NaN,20.0
dev-pa,v002,15.0,NaN,15.0
